In [1]:
import os
import sys


if "SUMO_HOME" in os.environ:
    tools = os.path.join(os.environ["SUMO_HOME"], "tools")
    sys.path.append(tools)
else:
    sys.exit("Please declare the environment variable 'SUMO_HOME'")
import numpy as np
import pandas as pd
import ray
import traci
from ray import tune
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
from ray.tune.registry import register_env

import sumo_rl


if __name__ == "__main__":
    # Use:
    # ray[rllib]==2.7.0
    # numpy == 1.23.4
    # Pillow>=9.4.0
    # ray[rllib]==2.7.0
    # SuperSuit>=3.9.0
    # torch>=1.13.1
    # tensorflow-probability>=0.19.0
    ray.init()

    env_name = "4x4grid"

    register_env(
        env_name,
        lambda _: ParallelPettingZooEnv(
            sumo_rl.parallel_env(
                net_file="4x4.net.xml",
                route_file="4x4c1c2c1c2.rou.xml",
                out_csv_name="outputs/4x4grid/ppo",
                use_gui=False,
                num_seconds=1000,
            )
        ),
    )

    config = (
        PPOConfig()
        .environment(env=env_name, disable_env_checking=True)
        .env_runners(num_env_runners=4, rollout_fragment_length=128)
        .training(
            train_batch_size=512,
            lr=2e-5,
            gamma=0.95,
            lambda_=0.9,
            use_gae=True,
            clip_param=0.4,
            grad_clip=None,
            entropy_coeff=0.1,
            vf_loss_coeff=0.25,
            minibatch_size=64,
            num_sgd_iter=10,
        )
        .debugging(log_level="ERROR")
        .framework(framework="torch")
        .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
    )

    tune.run(
        "PPO",
        name="PPO",
        stop={"timesteps_total": 100},
        checkpoint_freq=10,
        storage_path="~/ray_results/" + env_name,
        config=config.to_dict(),
    )

2025-02-17 13:41:55,724	WARNING compression.py:16 -- lz4 not available, disabling sample compression. This will significantly impact RLlib performance. To install lz4, run `pip install lz4`.
2025-02-17 13:42:01,423	INFO worker.py:1841 -- Started a local Ray instance.
2025-02-17 13:42:04,357	WARNING deprecation.py:50 -- DeprecationWarning: `config.training(num_sgd_iter=..)` has been deprecated. Use `config.training(num_epochs=..)` instead. This will raise an error in the future!
2025-02-17 13:42:04,360	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
c:\Python312\Lib\site-packages\gymnasium\spaces\box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
c:\Python312\Lib\site-packages\gymnasium\spaces\box.py:305: UserWarning: WARN: Box high's prec

2025-02-17 13:42:04,473	WARNING algorithm_config.py:4726 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
2025-02-17 13:42:04,473	WARNING algorithm_config.py:4726 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
(pid=25268) lz4 not available, disabling sample compression. This will significantly impact RLlib performance. To install lz4, run `pip install lz4`.
(PPO pid=25268) 2025-02-

(SingleAgentEnvRunner pid=10236) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 20ms, vehicles TOT 0 ACT 0 BUF 0)                     


(SingleAgentEnvRunner pid=20540) 2025-02-17 13:42:19,199	WARNING rl_module.py:419 -- Could not create a Catalog object for your RLModule! If you are not using the new API stack yet, make sure to switch it off in your config: `config.api_stack(enable_rl_module_and_learner=False, enable_env_runner_and_connector_v2=False)`. All algos use the new stack by default. Ignore this message, if your RLModule does not use a Catalog to build its sub-components.
(SingleAgentEnvRunner pid=20540) 2025-02-17 13:42:19,200	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!
(SingleAgentEnvRunner pid=20540) 2025-02-17 13:42:19,200	WARNING deprecation.py:50 -- DeprecationWarning: `get_rl_module_config` has been deprecated. Use `RLModule(*, observation_space=.., action_space=.., ....)` instead

Trial name
PPO_4x4grid_e1e2f_00000


2025-02-17 13:42:19,968	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'C:/Users/shrof/ray_results/4x4grid/PPO' in 0.0156s.
(SingleAgentEnvRunner pid=10236) Error: tcpip::Socket::recvAndCheck @ recv: Socket reset by peer


(SingleAgentEnvRunner pid=10236) Step #0.00


TuneError: ('Trials did not complete', [PPO_4x4grid_e1e2f_00000])

(SingleAgentEnvRunner pid=24224) Step #0.00


(SingleAgentEnvRunner pid=18736) Error: tcpip::Socket::recvAndCheck @ recv: Socket reset by peer
(SingleAgentEnvRunner pid=18736) Quitting (on error).
